https://qiita.com/ytaki0801/items/fc785de02eb45cde94fa

In [1]:
fib21 = 
[[["lambda", ["u"], ["u", "u"]],
        ["lambda", ["u"],
            ["lambda", ["n", "a", "b"],
                ["if", ["=", "n", 0], "a",
                    [["u", "u"], ["-", "n", 1], "b", ["+", "a", "b"]]]]]],
    21, 0, 1]

g = Dict("=" => "==", "+" => "+", "-" => "-")
function ev(s, e)
    s isa String     && return merge(g, e)[s]
    s isa Int        && return s
    s[1] == "if"     && return ev(s[2], e) ? ev(s[3], e) : ev(s[4], e)
    s[1] == "lambda" && return vcat(s, [e])
    f, a = ev(s[1], e), ev.(s[2:end], Ref(e))
    f isa String     && return eval(Expr(:call, Symbol(f), a...))
    return ev(f[3], merge(f[4], Dict(zip(f[2], a))))
end

@show ev(fib21, Dict());

ev(fib21, Dict()) = 10946


In [2]:
fib21 = 
[[["lambda", ["u"], ["u", "u"]],
        ["lambda", ["u"],
            ["lambda", ["n", "a", "b"],
                ["if", ["=", "n", 0], "a",
                    [["u", "u"], ["-", "n", 1], "b", ["+", "a", "b"]]]]]],
    21, 0, 1]

g = Dict("=" => ==, "+" => +, "-" => -)
function ev(s, e)
    s isa String     && return merge(g, e)[s]
    s isa Int        && return s
    s[1] == "if"     && return ev(s[2], e) ? ev(s[3], e) : ev(s[4], e)
    s[1] == "lambda" && return vcat(s, [e])
    f, a = ev(s[1], e), ev.(s[2:end], Ref(e))
    f isa Function   && return f(a...)
    return ev(f[3], merge(f[4], Dict(zip(f[2], a))))
end

@show ev(fib21, Dict());

ev(fib21, Dict()) = 10946


In [3]:
fib21 = 
[[[:lambda, [:u], [:u, :u]],
        [:lambda, [:u],
            [:lambda, [:n, :a, :b],
                [:if, [:(=), :n, 0], :a,
                    [[:u, :u], [:(-), :n, 1], :b, [:(+), :a, :b]]]]]],
    21, 0, 1]

g = Dict(:(=) => ==, :(+) => +, :(-) => -)
function ev(s, e)
    s isa Symbol     && return merge(g, e)[s]
    s isa Int        && return s
    s[1] === :if     && return ev(s[2], e) ? ev(s[3], e) : ev(s[4], e)
    s[1] === :lambda && return vcat(s, [e])
    f, a = ev(s[1], e), ev.(s[2:end], Ref(e))
    f isa Function   && return f(a...)
    return ev(f[3], merge(f[4], Dict(zip(f[2], a))))
end

@show ev(fib21, Dict());

ev(fib21, Dict()) = 10946
